In [239]:
import pandas as pd
import random
import numpy as np
import networkx as nx
import nltk
import tensorflow as tf
import collections
import math
from six.moves import xrange
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import MiniBatchKMeans

In [8]:
from functools import partial

class Infix(object):
    def __init__(self, func):
        self.func = func
    def __or__(self, other):
        return self.func(other)
    def __ror__(self, other):
        return Infix(partial(self.func, other))
    def __call__(self, v1, v2):
        return self.func(v1, v2)

## Read In

In [16]:
def load():
    ppp = nltk.data.load('../../Downloads/ppp.txt', encoding='utf8')
    words_p = nltk.tokenize.wordpunct_tokenize(ppp)[130:]
    alw = nltk.data.load('../../Downloads/alw.txt', encoding='utf8')
    words_a = nltk.tokenize.wordpunct_tokenize(alw)[143:]
    words = words_a + words_p
    return words

## Turn into list of Sentences

In [58]:
def get_sentences(words):
    sentences_word  = []
    sent_word = []
    for index, word in enumerate(words):
        if word in ['?','.','!']:
            sent_word += [word]
            sentences_word += [sent_word]
            sent_word = []
        else:
            sent_word += [word]
    return sentences_word

## Turn Vocab into Indices
-- This was process, but I think this is useful regardless. I will need to change it to accomodate ..... stuff?

In [371]:
def build_vocab(words, vocabulary_size):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, dictionary, reverse_dictionary

## Set Up Graph Building

Basically we want to make sure that things are being added to the graph. How do we do this. Well. I think we should take them as if they were read in line and then only add things like that. This way we can do all our learning right good without having to stop and add to the graph

Adding node likelihood : http://ieeexplore.ieee.org/document/7266560/

In [187]:
sim = vec_graph(KB, embeddings, [i for i in range(100)], 10000)

In [190]:
embeddings.shape

(15000, 100)

In [186]:
def vec_graph(KB, embeddings, sentence, max_nodes):
    vecsen = np.array(sentence)
    p_edges = embeddings[vecsen[1:]] - embeddings[vecsen[:-1]]
    sim = cosine_similarity(embeddings, p_edges, )
    
    return sim

In [102]:
def graph(KB, embeddings, sentence, max_nodes):
    def next_lvl(sentence):
        @Infix
        def to(a, b):
            x = embeddings[sentence[b]] - embeddings[sentence[a]]
            return x
        
        nl = []
        ln = []
        for index in range(len(sentence[:-1])):
            if KB.has_edge(sentence[index], sentence[index+1]):
                nl.append(KB[sentence[index]][sentence[index+1]]['node'])
            else:
                if KB.number_of_nodes()+1 < max_nodes:
                    KB.add_node(KB.number_of_nodes(), exp=.5)
                    KB.add_edge(sentence[index], sentence[index+1],
                               node=KB.number_of_nodes())
                    embeddings[KB.number_of_nodes()] = index |to| (index+1)
                else:
                    x = index |to| (index+1)
                    similarity = cosine_similarity(embeddings, [x])
                    nearest = np.argmax(similarity)
                    exp = KB.node[nearest]['exp']
                    embeddings[nearest] = embeddings[nearest]*exp + x*(1-exp)
                    KB.node[nearest]['exp'] += (1-exp)**3
                    if 1-similarity.max() > 1-exp:
                        nl.append(nearest)
                    
                    if not KB.has_edge(sentence[index], sentence[index+1]):
                        KB.add_edge(sentence[index], sentence[index+1], 
                                    node=nearest)
        return nl
    
    
    l_all = [sentence]
    l_next = sentence
    while l_next:
        l_next = next_lvl(l_next)
        print(l_next)
        l_all += l_next
    return KB, embeddings, l_all[:-1]

In [367]:
def prep_graph(words):
    KB2 = nx.Graph()
    for word in words:
        if not KB2.has_node(word):
            KB2.add_node(word)
        
    KB3 = nx.convert_node_labels_to_integers(KB2, label_attribute='word')
    KB4 = nx.Graph()
    
    for node in KB3.nodes(True):
        KB4.add_node(node[1]['word'], number=node[0])
    return KB4

## Main Function

## Generate Each Batch
-- a useful thing would be for batches to be made of few words, a couple sentences, and then be taken from there. idk.

In [337]:
data_index = 0
def gen_batch(data, batch_size, skip_window, num_skips):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1 # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [ skip_window ]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels


## Do the Actual Word2Vec Algorithm in TF

In [352]:
def W2V(batch_size, embedding_size, skip_window, num_skips, valid_size,
       valid_window, valid_examples, num_sampled, vocabulary_size,
       num_steps, data, revdic):
    graph = tf.Graph()
    
    with graph.as_default():
        train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
        train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
        valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
        
        with tf.device('/cpu:0'):
            
            embeddings = tf.Variable(
                            tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
            embed = tf.nn.embedding_lookup(embeddings, train_inputs)
        
        nce_weights = tf.Variable(
            tf.truncated_normal([vocabulary_size, embedding_size], 
                               stddev=1.0 / math.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
        
        loss = tf.reduce_mean(
            tf.nn.nce_loss(nce_weights, nce_biases, embed, train_labels,
                          num_sampled, vocabulary_size))
        optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)
        
        norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 
                                     1, keep_dims=True))
        normalized_embeddings = embeddings / norm
        valid_embeddings = tf.nn.embedding_lookup(
            normalized_embeddings, valid_dataset)
        similarity = tf.matmul(
            valid_embeddings, normalized_embeddings, transpose_b=True)
        
        init = tf.initialize_all_variables()
    
    with tf.Session(graph=graph) as session:
        
        init.run()
        print("Initialized")
        #saver = tf.train.Saver({'In'})
        
        average_loss = 0
        for step in xrange(num_steps):
            batch_inputs, batch_labels = gen_batch(
                data, batch_size, num_skips, skip_window)
            feed_dict = {train_inputs : batch_inputs, 
                         train_labels : batch_labels}
            
            _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
            average_loss += loss_val

        
            if step % 2000 == 0:
                if step > 0:
                    average_loss /= 2000
                # The average loss is an estimate of the loss over the last 2000 batches.
                print("Average loss at step ", step, ": ", average_loss)
                average_loss = 0

            # Note that this is expensive (~20% slowdown if computed every 500 steps)
            if step % 10000 == 0 and step > 0:
                sim = similarity.eval()
                for i in xrange(valid_size):
                    valid_word = revdic[valid_examples[i]]
                    top_k = 8 # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k+1]
                    log_str = "Nearest to %s:" % valid_word
                    for k in xrange(top_k):
                        close_word = revdic[nearest[k]]
                        log_str = "%s %s," % (log_str, close_word)
                    print(log_str)

        final_embeddings = normalized_embeddings.eval()
        return final_embeddings
        
    

In [368]:
num_read = 1000
def main():
    counts = collections.Counter()
    words = load()
    KB = prep_graph(words)
    
    data = []
    for word in words[:20000]:
        data.append(KB.node[word]['number'])
        counts.update(word)
    
    revdic = {node[1]['number']: node[0] for node in KB.nodes(True)}
    vocab_size = KB.number_of_nodes()

    batch_size = 128
    embedding_size = 100  # Dimension of the embedding vector.
    skip_window = 2      # How many words to consider left and right.
    num_skips = 4         # How many times to reuse an input to generate a label.
    
    # We pick a random validation set to sample nearest neighbors. Here we limit the
    # validation samples to the words that have a low numeric ID, which by
    # construction are also the most frequent.
    valid_size = 50    # Random set of words to evaluate similarity on.
    valid_window = 200  # Only pick dev samples in the head of the distribution.
    valid_examples = np.random.choice(valid_window, valid_size, replace=False)
    num_sampled = 70    # Number of negative examples to sample.
    num_steps = 10000
    
    data, dic, revdic = build_vocab(words, vocab_size)
    embeddings = W2V(batch_size, embedding_size, skip_window,
                    num_skips, valid_size, valid_window,
                    valid_examples, num_sampled, vocab_size,
                    num_steps, data, revdic)
    
#     KB = nx.DiGraph()
#     KB.add_nodes_from(list(dictionary.values()), exp=.95)
#     embeddings_2 = np.concatenate((embeddings, np.zeros((5000, 100))))
    
#     sentences = get_sentences(words)
#     data_2 = []
#     for sentence in sentences[:100]:
#         KB, embeddings_2, hlsent = graph(KB, embeddings_2, data, len(embeddings_2))
#         data_2 += hlsent
    
    counts, words = add_level(words[20000:30000], embeddings, KB, counts)
    
    return counts

data2 = main()

KeyError: 'Hole'

In [372]:
counts = collections.Counter()
words = load()

KB = prep_graph(words)

data = []
for word in words[:50000]:
    data.append(KB.node[word]['number'])
    counts.update(word)

revdic = {node[1]['number']: node[0] for node in KB.nodes(True)}
vocab_size = KB.number_of_nodes()

batch_size = 128
embedding_size = 100  # Dimension of the embedding vector.
skip_window = 2      # How many words to consider left and right.
num_skips = 4         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 50    # Random set of words to evaluate similarity on.
valid_window = 200  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 70    # Number of negative examples to sample.
num_steps = 20000

data, dic, revdic = build_vocab(words, vocab_size)
embeddings = W2V(batch_size, embedding_size, skip_window,
                num_skips, valid_size, valid_window,
                valid_examples, num_sampled, vocab_size,
                num_steps, data, revdic)

#     KB = nx.DiGraph()
#     KB.add_nodes_from(list(dictionary.values()), exp=.95)
#     embeddings_2 = np.concatenate((embeddings, np.zeros((5000, 100))))

#     sentences = get_sentences(words)
#     data_2 = []
#     for sentence in sentences[:100]:
#         KB, embeddings_2, hlsent = graph(KB, embeddings_2, data, len(embeddings_2))
#         data_2 += hlsent



Initialized
Average loss at step  0 :  225.932037354
Average loss at step  2000 :  36.3504361038
Average loss at step  4000 :  7.90283696294
Average loss at step  6000 :  5.65475662613
Average loss at step  8000 :  5.25224653077
Average loss at step  10000 :  5.12791293502
Nearest to so: looked, oldest, one, prefer, ordered, door, me, downstairs,
Nearest to their: B, glance, Louisa, straws, between, dependence, shore, unlikely,
Nearest to made: compliance, unfolded, rose, the, masters, cold, daisies, appearance,
Nearest to great: impropriety, dispose, bore, one, belt, MERCHANTIBILITY, distress, waived,
Nearest to can: dead, ,, incredulous, pale, rule, profession, mission, rudeness,
Nearest to ,’: Had, loss, scorned, whispered, Mary, respectable, begged, thankfulness,
Nearest to Mrs: instead, expedient, stretch, vivacity, Both, however, de, seldom,
Nearest to Lady: omit, themselves, mark, distributed, gave, announce, discussed, fretfully,
Nearest to one: permit, compliment, prefer, blin

In [388]:
words_n, KB = add_level(words[:40000], embeddings, KB, counts)

/home/lenny/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:1267: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  distances = np.zeros(self.batch_size, dtype=np.float64)
/home/lenny/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:1279: DeprecationWarning: This function is deprecated. Please call randint(0, 39998 + 1) instead
  0, n_samples - 1, init_size)
/home/lenny/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:1279: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  0, n_samples - 1, init_size)
/home/lenny/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:630: DeprecationWarning: This function is deprecated. Please call randint(0, 39998 + 1) instead
  0, n_samples - 1, init_size)
/home/lenny/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:630: VisibleDeprecationWar

In [392]:
revdic = {node[1]['number']: node[0] for node in KB.nodes(True)}
words =  words_n + words[40000:]
data = [KB.node[word]['number'] for word in words]

In [394]:
batch_size = 256
embedding_size = 200  # Dimension of the embedding vector.
skip_window = 4      # How many words to consider left and right.
num_skips = 8         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 500    # Random set of words to evaluate similarity on.
valid_window = 1000  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 200    # Number of negative examples to sample.
num_steps = 40000

In [395]:
vocab_size = KB.number_of_nodes()
data_index=0
embeddings = W2V(batch_size, embedding_size, skip_window,
                num_skips, valid_size, valid_window,
                valid_examples, num_sampled, vocab_size,
                num_steps, data, revdic)


Initialized
Average loss at step  0 :  583.893554688
Average loss at step  2000 :  43.12431568
Average loss at step  4000 :  15.7613292387
Average loss at step  6000 :  5.31909964597
Average loss at step  8000 :  4.01613262784
Average loss at step  10000 :  4.37640485317
Nearest to slept: unimportant, 8941, chair, 10517, 10380, 11810, serpents, clock,
Nearest to skirt: ,, awoke, e, involve, despair, church, Drawling, BUT,
Nearest to delay: Tortoise, welfare, Seven, allayed, thoughtless, 9163, defiance, arranged,
Nearest to hoarse: governess, indirect, ,, Tease, evenings, M, contact, Drawling,
Nearest to teeth: poverty, experiment, Drawling, defiance, offense, --‘, evenings, danced,
Nearest to behaves: church, 9185, jury, cake, 10261, vulgar, conger, verse,
Nearest to maximum: countries, 11037, 9036, concept, PROJECT, FROM, manifold, peeped,
Nearest to honestly: This, 8975, Distraction, faces, inevitable, passion, tempt, exclaiming,
Nearest to DAMAGE: officer, 9900, 9866, be, scratching

In [387]:
def add_level(words, embeddings, KB, counts):
    data = [KB.node[word]['number'] for word in words]
    
    embed_data = np.array([embeddings[wordnum] for wordnum in data])
    
    next_lvl_raw = embed_data[1:] - embed_data[:-1]
    next_lvl_cent = TFKMC(next_lvl_raw, n_clusters=500)
    
    k = KB.number_of_nodes()
    words_n = [words[0]]
    vocab_size = KB.number_of_nodes()
    
    for num in range(vocab_size, vocab_size+1000):
        KB.add_node(str(num), number=num)
    
    for i in range(len(next_lvl_cent.labels_)):
        t = next_lvl_cent.labels_[i]
        words_n.extend([str(t+vocab_size), words[i+1]])
        KB.add_edge(words[i], words[i+1], node=str(t))
        
    
    
    return words_n, KB     
    
    
    

In [370]:
def TFKMC(vectors, n_clusters=1000, max_iter=100000):
    mbatch = MiniBatchKMeans(n_clusters=n_clusters, batch_size=len(vectors)*.1, max_iter=max_iter)
    centroids = mbatch.fit(vectors)
    return centroids

## What To Do

### 1. Read Gradient Descent Site
### 2. Read the Candidate Sampling Thing
### 3. Implement KMeans MiniBatch in TensorFlow
### 4. Write Graph Adding Code
### 5. Write Processing for Transitions into Graph
### 6. Implement Word2Vec Using Graphs
